In [ ]:
!pip install rasterio==1.3.9

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import csv
import json
import matplotlib.pyplot as plt
import folium

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# GeoTiff Images
import rasterio
from osgeo import gdal

# Visualisation
from matplotlib import pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure
from PIL import Image

"""# Model Building
import ultralytics
from ultralytics import YOLO
import labelme2yolo"""

# Others
import os
import shutil
import zipfile

from tqdm import tqdm
import glob

# Create a dataset of building footprint segmentation masks. 

with S3 credential and SSP Cloud access : 
- mc cp s3/mbesnier/damage_detection/building_footprints_puerto_rico ./ --recursive 

Task Description : 
- [x] Finding the corresponding files containing the building footprints in Puerto Rico
- [x] Extracting the footprints with geolocated information
- [x] Associate the titles from the data processing of Puerto Rico (images from tiff files) to the geolocalized masks. (use a KD tree) (for each retrieved footprint find the closest title centroid )
- [ ] Transform the building footprint into a segmentation mask for semantic segmentation instance. 
- [ ] Build the dataset: image/masks => Train/Eval/Test dataset
- [ ] Train a Building detector (use **Active learning** to know which extra data should be annotated. )
- [ ] Apply some technics from conformal prediction to evaluate Classifier and Segmentation model.   
- [ ] Train an **Image segmentation** Building Footprint detector (with and without extra classes => See the transfer learning possibilities).

In [ ]:
list_paths = glob.glob("/home/onyxia/work/building_footprints_puerto_rico/*.csv") #put the files of footprint
print(len(list_paths))

In [ ]:
def safe_json_loads(data):
    try:
        return json.loads(data)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print("line : ", data)
        return None

In [ ]:
#put the files of footprint
import json

# reading the data from the file
json_objects = []
for path in list_paths:
      print("processing " + str(path)) 
      with open(path) as f:
          data = f.read()
  # reconstructing the data as a dictionary
      json_objects.extend([safe_json_loads(line) for line in data.strip().split('\n')])

In [ ]:
len(json_objects)

In [ ]:
import pandas as pd
from shapely.geometry import Polygon

parsed_data = [Polygon(item['geometry']['coordinates'][0]) for item in json_objects if item is not None ]
gdf = gpd.GeoDataFrame(data=parsed_data, columns=['geometry'])
gdf.set_crs("EPSG:4326", inplace=True)

# Test

In [ ]:
import random
gdf = gdf.sample(n=1000, random_state=42)

In [ ]:
gdf.geometry.to_crs("4326")
centroid = gdf.geometry.centroid #centroids of the bulding footprints. 
center =  [centroid.y.mean(), centroid.x.mean()]
map = folium.Map(location=center, zoom_start=8) #Microsoft use this geodesic referential : EPSG: 4326 (correspond to GPS)

# Access to EY Challenge Data (Pre event and Post event San Juan catastrophy)

In [ ]:
!mc cp s3/mbesnier/damage_detection/raw_data . --recursive  #load the data from the S3 bucket

In [ ]:
# load the data from API 
!wget "https://challenge.ey.com/api/v1/storage/admin-files/Pre_Event_San_Juan.tif" -O "Pre_Event_San_Juan.tif"
!wget "https://challenge.ey.com/api/v1/storage/admin-files/Post_Event_San_Juan.tif" -O "Post_Event_San_Juan.tif"

In [ ]:
pre_event_image = './Pre_Event_San_Juan.tif'

In [ ]:
gdf.bounds.head()

# Generate Tiles from raw tiff pictures

In [ ]:
def generate_tiles(input_file, output_dir,grid_x,grid_y):
    ds = gdal.Open(input_file)

    # Get image size and number of bands
    width = ds.RasterXSize
    height = ds.RasterYSize
    num_bands = ds.RasterCount

    # Calculate number of tiles in each dimension
    num_tiles_x = (width // grid_x)
    num_tiles_y = (height // grid_y)

    print(f"Total number of tiles: {num_tiles_x * num_tiles_y}")

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterate over each tile and save as a separate TIFF image
    for i in tqdm(range(num_tiles_x)):
        for j in range(num_tiles_y):
            x_offset = i *  grid_x
            y_offset = j *  grid_y

            tile_width = min(grid_x, width - x_offset)
            tile_height = min(grid_y, height - y_offset)

            tile = []
            for band in range(1, num_bands + 1):
                tile_data = ds.GetRasterBand(band).ReadAsArray(x_offset, y_offset, tile_width, tile_height)
                tile.append(tile_data)

            # Create output filename
            output_file = os.path.join(output_dir, f"tile_{i}_{j}.tif")

            # Create an output TIFF file with same CRS and band values range
            driver = gdal.GetDriverByName("GTiff")
            options = ['COMPRESS=DEFLATE', 'PREDICTOR=2', 'TILED=YES']
            out_ds = driver.Create(output_file, tile_width, tile_height, num_bands,
                       ds.GetRasterBand(1).DataType, options=options)
            # out_ds = driver.Create(output_file, tile_width, tile_height, num_bands, ds.GetRasterBand(1).DataType)

            # Set the geotransform
            geotransform = list(ds.GetGeoTransform())
            geotransform[0] = geotransform[0] + x_offset * geotransform[1]
            geotransform[3] = geotransform[3] + y_offset * geotransform[5]
            out_ds.SetGeoTransform(tuple(geotransform))

            # Set the projection
            out_ds.SetProjection(ds.GetProjection())

            # Write each band to the output file
            for band in range(1, num_bands + 1):
                out_band = out_ds.GetRasterBand(band)
                out_band.WriteArray(tile[band - 1])

            # Close the output file
            out_ds = None

    print("Tiles generation completed.")

In [ ]:
input_file = "Post_Event_San_Juan.tif"
output_dir = "./Post_Event_Grids_In_TIFF"
grid_x = 512
grid_y = 512
generate_tiles(input_file, output_dir, grid_x, grid_y)

In [ ]:
input_file = "Pre_Event_San_Juan.tif"
output_dir = "./Pre_Event_Grids_In_TIFF"
grid_x = 512
grid_y = 512
generate_tiles(input_file, output_dir, grid_x, grid_y)

In [ ]:
# Open the geotiff file
import re 
def extract_title_ij(filename):
    pattern = r"(\w+)/tile_(\d+)_(\d+).tif"
    match = re.search(pattern, filename)
    if match:
        title , i, j = map(str, match.groups())
        return title, i, j
    else:
        return None, None, None


def load_and_visualize(image_path):
    """
    display a tif image (little tiles are recommended)
    """
    with rasterio.open(image_path) as src:
        # Read the red, green, and blue bands directly into a 3D array
        image_rgb = src.read([1, 2, 3])  # Read bands 1, 2, and 3 in a single call
        # Transpose the array for correct display (optional)
        image_rgb = image_rgb.transpose(1, 2, 0)

        title , i, j = extract_title_ij(image_path)

        # Display the RGB composite image
        plt.imshow(image_rgb)
        plt.title(f"Image from {title} at position (i={i},j={j})")
        plt.show()

In [ ]:
i = 8 
j = 100

load_and_visualize(f"Pre_Event_Grids_In_TIFF/tile_{i}_{j}.tif")
load_and_visualize(f"Post_Event_Grids_In_TIFF/tile_{i}_{j}.tif")

In [ ]:
import shapely.geometry
import rasterio.features 

def extract_coor_from_tiff_image(path):
  """
  extract coordinates associated to a tif image
  """
  with rasterio.open(path) as dataset:

    # Read the dataset's valid data mask as a ndarray.

    mask = dataset.dataset_mask()

    # Extract feature shapes and values from the array.
    for geom, val in rasterio.features.shapes(
            mask, transform=dataset.transform):
        
        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326).
        geom = rasterio.warp.transform_geom(
            dataset.crs, 'EPSG:4326', geom, precision=6)

        bounds = shapely.geometry.shape(geom).bounds
        #minx, miny, maxx, maxy = bounds
        bounds = [[bounds[1], bounds[0]], [bounds[3], bounds[2]]]
        #[[18.461885, -66.190829], [18.463317, -66.189328]]
        # lat_min, 

        return geom, bounds # return a geojson object (polygon localized on a map)
        #bounds are folium compatible

def convert_tiff_to_numpy(tif_path):
    # check if output_dir exists, if not create it
    # check if file is an image (ends with .tif)
    with rasterio.open(tif_path)as src:
          image_array = src.read([1, 2, 3])  # Read bands 1, 2, and 3 in a single call
          # Transpose the array for correct display (optional)
          image_array = image_array.transpose(1, 2, 0)
          return image_array    


In [ ]:
extract_coor_from_tiff_image("Pre_Event_Grids_In_TIFF/tile_9_86.tif")

In [ ]:
import multiprocessing as mp
from joblib import Parallel, delayed

def process_tif(path):
    geom , bounds = extract_coor_from_tiff_image(path)
    geom = Polygon(geom["coordinates"][0])
    return [path, geom]

def table_tiff_geojson_fast(folder_tiff):
    """
    create a geo dataframe from a tiff folder
    """
    if os.path.exists(folder_tiff):
        list_tifs = glob.glob(folder_tiff + "/*.tif")
        gdf = gpd.GeoDataFrame(columns=["path", "geometry"], index=range(len(list_tifs)))

        num_cores = mp.cpu_count()  # Number of cores in your machine
        print("number of cores : ", num_cores)
        results = Parallel(n_jobs=num_cores)(delayed(process_tif)(path) for path in list_tifs)

        for i, result in enumerate(results):
            gdf.loc[i] = result

        print(f"{folder_tiff} has been converted into GeoJSON DataFrame")
        return gdf

In [ ]:
def table_tiff_geojson(folder_tiff):
    """
    create a geo dataframe from a tiff folder
    """
    if os.path.exists(folder_tiff):
      list_tifs = glob.glob(folder_tiff + "/*.tif")

      gdf = gpd.GeoDataFrame(columns=["path", "geometry"])
      for i, path in tqdm(enumerate(list_tifs)):
        geom , bounds = extract_coor_from_tiff_image(path)
        geom = Polygon(geom["coordinates"][0])
        gdf.loc[len(gdf)] = [list_tifs[i], geom] #geom ESPG 4326
    print(f"{folder_tiff} has been converted into GeoJSON DataFrame")
    return gdf

In [ ]:
%%time
folder_tiff = "Pre_Event_Grids_In_TIFF"
table_tiff_pre = table_tiff_geojson_fast(folder_tiff) #table_tiff_geojson(folder_tiff)

In [ ]:
%%time
folder_tiff = "Post_Event_Grids_In_TIFF"
table_tiff_post = table_tiff_geojson_fast(folder_tiff)

In [ ]:
print("Pre events :  ", len(table_tiff_pre))
print("Post events :  ", len(table_tiff_post))

In [ ]:
#qurey points are the centroids of the footprint building gdf["centroids"]
from sklearn.neighbors import KDTree
import time 
from shapely import Point
#find the images where foot prints appears

def matching_footprint_to_tiff_images(ft_gdf, table_tiff):
      """
      Matching footprints with tiff images (by using centroids)
      """
      t0 = time.time()
      required_cols = ["geometry"]
      if not all(col in ft_gdf.columns for col in required_cols):
        raise ValueError("Missing required columns in ft_gdf.")
      if not all(col in table_tiff.columns for col in required_cols):
        raise ValueError("Missing required columns in table_tiff.")

      # to compute distance btw these 2 figures they need to be put into the same geodesic referential system (GPS : lat, long)
      ft_gdf = ft_gdf.set_crs('epsg:4326') #Geodesic coordinate
      table_tiff = table_tiff.set_crs('epsg:4326')

      #to use euclidian distance
      ft_gdf = ft_gdf.to_crs('EPSG:3857') #EPSG:3857 Projected coordinate Mercator (Map)
      table_tiff = table_tiff.to_crs('EPSG:3857')

      centroid_query = np.array(list(zip(ft_gdf.centroid.x.to_numpy() , ft_gdf.centroid.y.to_numpy())))
      centroid_table = np.array(list(zip(table_tiff.centroid.x.to_numpy() , table_tiff.centroid.y.to_numpy())))

      print("creation of the KD Tree for tile centroid")
      print("number of footprints : ", len(centroid_query))
      print("number of titles : ", len(centroid_table))
        
      tree = KDTree(centroid_table, leaf_size = 20, metric='euclidean')

      print("query the Tree with the centroid of the footprints")
      distances, indices = tree.query(centroid_query, k=1) #each footprint centroid find its tile by finding the nearest tile centroid
      response = [str(table_tiff.iloc[i]["path"]) for i in indices.squeeze(1)]
      tile_centroids = [Point(centroid_table[i]) for i in indices.squeeze(1)]

      df = {}

      ft_gdf = ft_gdf.to_crs('epsg:4326')
      df["path_to_image"] = pd.Series(response) # associated images 
      df["distance"] = pd.Series(distances[:,0]) #in meters
      df["geometry"] = ft_gdf["geometry"].reset_index(drop=True) #associated footprints 
      df["centroid_tile"] = gpd.GeoSeries(tile_centroids, crs='EPSG:3857').to_crs('epsg:4326')
        
      print("processing time : ", time.time() - t0, "s")
      return gpd.GeoDataFrame(df)

In [ ]:
matching_table_pre = matching_footprint_to_tiff_images(gdf, table_tiff_pre)

In [ ]:
matching_table_post = matching_footprint_to_tiff_images(gdf, table_tiff_post)

Analysis : The number of footprint is much higher than the number of houses in the tiff images since the available footprints are located all across the Puerto Rico Island. We have split the images into 150x150 m square chuncks thus we only select the footprint located at most at 75m of distance from the centroid tile. 

In [ ]:
matching_table_post[matching_table_post["distance"] < 75].describe() # 580 866 rows

The building footrpints located at less than 75 meters from their associated tile centroid

In [ ]:
m = gpd.GeoDataFrame(geometry=matching_table[matching_table["distance"] < 75]["geometry"]).explore(color="blue", name="footprint")
m = gpd.GeoDataFrame(geometry=matching_table[matching_table["distance"] < 75]["centroid_tile"]).explore(m=m, color="red",name="tile_centroids")
folium.LayerControl().add_to(m)
m

In [ ]:
dist = 75
#pre event footprint extraction 
dataset_pre = matching_table_pre[matching_table_pre["distance"] < dist] #43169 footprints are inside the San_Juan data set 
dataset_pre.to_csv("Pre_event_dataset_EY_path_dist_geometry_" + str(dist) + "m.csv", index = False)

dataset_post = matching_table_post[matching_table_post["distance"] < dist] #43169 footprints are inside the San_Juan data set 
dataset_post.to_csv("Post_event_dataset_EY_path_dist_geometry_" + str(dist) + "m.csv", index = False)

In [ ]:
def calculate_distance_from_tiff(path):
    with rasterio.open(path) as dataset:
        # Get pixel resolution (size of each pixel)
        x_resolution, y_resolution = dataset.res

        # Get image dimensions (number of pixels in each dimension)
        width = dataset.width
        height = dataset.height

        # Calculate distance covered in each dimension
        distance_x = width * x_resolution
        distance_y = height * y_resolution

        return distance_x, distance_y


In [ ]:
def data_gathering(data):
    """
    from a filtered dataset with pair (image, footprint) convert the images into array
    """
    list_path = data.path_to_image.unique()
    res = {}
    for path in tqdm(list_path):
         output = data[data["path_to_image"] == path]
         res[path] = output.geometry.array
    return res

results_pre = data_gathering(dataset_pre)
results_post = data_gathering(dataset_post)

In [ ]:
def convert_tiff_to_jpeg(input_dir,output_dir):
    """
    Converting tiff images to jpeg images.
    """
    # check if output_dir exists, if not create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        # check if file is an image (ends with .tif)
        if filename.endswith('.tif'):
            img = Image.open(os.path.join(input_dir, filename))
        
            # check if image is RGB mode, if not convert it
            if img.mode != 'RGB':
                img = img.convert('RGB')
        
            # create new filename, replace .tif with .jpg
            output_filename = os.path.splitext(filename)[0] + '.jpg'
        
            # save the image in JPEG format
            img.save(os.path.join(output_dir, output_filename), 'JPEG', quality=100)
    print("Conversion from TIFF to JPEG completed.")

In [ ]:
from rasterio.mask import mask
from rasterio.plot import show
import geopandas as gdp

image_path = np.random.choice(list(results_pre.keys()))
info = results_pre[image_path] #extract the associated building footprint
print(image_path , " : ", info)

with rasterio.open(image_path) as src:
        # Get the CRS (Coordinate Reference System) of the TIFF image
        crs = src.crs
        polygons = gdp.GeoDataFrame(geometry = info) 
        polygons = polygons.to_crs(crs)
        # Create subplot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))

        # Plot RGB image
        show(src, ax=ax1)
        ax1.set_title('RGB Image')
        
        # Plot building footprint
        polygons.plot(ax=ax2)
        ax2.set_title('Building Footprint Polygon')

        plt.show()


In [ ]:

image_path = "Post_Event_Grids_In_TIFF/tile_71_73.tif"
with rasterio.open(image_path) as src:
        # Get the CRS (Coordinate Reference System) of the TIFF image
        crs = src.crs
        polygons = gdp.GeoDataFrame(geometry = info) 
        polygons = polygons.to_crs(crs)
        # Create subplot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))

        # Plot RGB image
        show(src, ax=ax1)
        ax1.set_title('RGB Image')
        
        # Plot building footprint
        polygons.plot(ax=ax2)
        ax2.set_title('Building Footprint Polygon')

        plt.show()


# Display RGB image with Building Footprint

In [ ]:
geom , bounds = extract_coor_from_tiff_image(image_path)
print(bounds)
polygons_ = info

# Filter out empty or missing polygons
valid_polygons = [polygon for polygon in polygons_ if not polygon.is_empty]

# Create the map
m = folium.Map(location=bounds[0], zoom_start=15)

# Add valid polygons to the map
bbox = folium.GeoJson(
    data={"type": "FeatureCollection", "features": [
        {"type": "Feature", "geometry": shapely.geometry.mapping(polygon)} for polygon in valid_polygons
    ]},
    style_function=lambda x: {'color': 'blue'}
).add_to(m)

# Add the image overlay
img = folium.raster_layers.ImageOverlay(
    name="pre event Puerto Rico",
    bounds=bounds,
    image=convert_tiff_to_numpy(image_path),
    opacity=1,
    cross_origin=False,
    zindex=1
).add_to(m)

# Add a popup
folium.Popup("pre event Puerto Rico").add_to(m)

# Show the map
m


In [ ]:
from shapely.ops import cascaded_union
from rasterio.plot import reshape_as_image
from rasterio import features
import re 

#Generate polygon
def poly_from_utm(polygon, transform):
    poly_pts = []

    poly = cascaded_union(polygon)
    for i in np.array(poly.exterior.coords):

        # Convert polygons to the image CRS
        poly_pts.append(~transform * tuple(i))

    # Generate a polygon object
    new_poly = Polygon(poly_pts)
    return new_poly

    
def generate_mask(image_path, shapes, save_path_o):
    """
    image_path : string 
    shapes : list of polygones associated to image_path 
    """ 

    if not os.path.exists(save_path_o):
        os.makedirs(save_path_o)

    with rasterio.open(image_path, "r") as src: #read tiff image

        raster_image = src.read()
        raster_meta = src.meta
        crs_image = raster_meta["crs"]

    train_df = gpd.GeoDataFrame(geometry = shapes).to_crs(crs_image) #create a geo pandas with the footprints 

    poly_shp = []
    im_size = (src.meta['height'], src.meta['width'])
    for num, row in train_df.iterrows():
        if row['geometry'].geom_type == 'Polygon':
            poly = poly_from_utm(row['geometry'], src.meta['transform'])
            poly_shp.append((poly, num))
        else:
            for p in row['geometry']:
                poly = poly_from_utm(p, src.meta['transform'])
                poly_shp.append((poly,num))
    
    #create a mask from the footprint
    mask = features.rasterize(shapes=poly_shp,
                     out_shape=im_size, 
                     fill = -1,
                     
                    )

    """# Create subplot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))

    # Plot RGB image
    ax1.imshow(image.read().transpose(1, 2, 0))
    ax1.set_title('RGB Image')

    # Plot building footprint
    ax2.imshow(mask)
    ax2.set_title('Building Footprint Mask')

    plt.show();
    """

    delimiters = "/. "
    mask = mask.astype("uint16")

    save_path = save_path_o + "/" + re.split('[{}]+'.format(''.join(map(re.escape, delimiters))), image_path)[1] + "_mask.tif"
    bin_mask_meta = src.meta.copy()
    bin_mask_meta.update({'count': 1})
    with rasterio.open(save_path, 'w', **bin_mask_meta) as dst:
        dst.write(mask * 255, 1)


In [ ]:
%%time
#converting tiff images into JPEG images
convert_tiff_to_jpeg(input_dir = "Pre_Event_Grids_In_TIFF", output_dir = "Pre_Event_Grids_In_jpeg")

In [ ]:
convert_tiff_to_jpeg(input_dir = "Post_Event_Grids_In_TIFF", output_dir = "Post_Event_Grids_In_jpeg")

In [ ]:
convert_tiff_to_jpeg(input_dir = "Pre_Event_Grids_In_TIFF_mask",output_dir="Pre_Event_Grids_In_TIFF_mask_jpeg")

In [ ]:
def convert_tiff_images_to_jpeg(image_paths_tiff, output_dir="Pre_Event_Grids_In_jpeg"):
    # check if output_dir exists, if not create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in tqdm(image_paths_tiff):
        # check if file is an image (ends with .tif)
        if filename.endswith('.tif'):
            img = Image.open(os.path.join(filename))
        
            # check if image is RGB mode, if not convert it
            if img.mode != 'RGB':
                img = img.convert('RGB')
        
            # create new filename, replace .tif with .jpg
            delimiters = "/. "
            output_filename = re.split('[{}]+'.format(''.join(map(re.escape, delimiters))), filename)[1] + '.jpg'
        
            # save the image in JPEG format
            img.save(os.path.join(output_dir, output_filename), 'JPEG', quality=100)
    print("Conversion from TIFF to JPEG completed.")

In [ ]:
#converting the selected images to JPEG format 
convert_tiff_images_to_jpeg(results_pre.keys(),"Pre_Event_Grids_In_jpeg") 

In [ ]:
#converting the selected images to JPEG format 
convert_tiff_images_to_jpeg(results_post.keys(),"Post_Event_Grids_In_jpeg") 

In [ ]:
#generate the masks
save_path_o = "Pre_Event_Grids_In_TIFF_mask" # "Post_Event_Grids_In_TIFF_mask"

for image_path in tqdm(results_pre.keys()):
    generate_mask(image_path, results_pre[image_path], save_path_o)

In [ ]:
save_path_o = "Post_Event_Grids_In_TIFF_mask"

for image_path in tqdm(results_post.keys()):
    generate_mask(image_path, results_post[image_path], save_path_o)


# Converting 

In [ ]:
convert_tiff_to_jpeg(input_dir="Pre_Event_Grids_In_TIFF_mask",output_dir="Pre_Event_Grids_mask_jpeg")

In [ ]:
convert_tiff_to_jpeg(input_dir="Post_Event_Grids_In_TIFF_mask",output_dir="Post_Event_Grids_mask_jpeg")

Compressing mask folder is not mandatory

In [ ]:
import shutil

# Specify the directory you want to zip
dir_name = 'Pre_Event_Grids_In_TIFF_mask_jpeg'

# Create a zip archive
output_filename = 'Pre_Event_Grids_In_TIFF_mask_jpeg'
shutil.make_archive(output_filename, 'zip', dir_name)

In [ ]:
# Specify the directory you want to zip
dir_name = 'Post_Event_Grids_In_TIFF_jpeg'

# Create a zip archive
output_filename = 'Post_Event_Grids_In_TIFF_jpeg'
shutil.make_archive(output_filename, 'zip', dir_name)